<a href="https://colab.research.google.com/github/Elma-dev/pySpark_Exercices/blob/main/pySparkExercice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425344 sha256=f6de37ccd917173cf59b242d197cfa07199a60796e303e38ee39cb933ab309e7
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
from pyspark.context import SparkContext
sc=SparkContext("local","test")

In [3]:
fileRdd=sc.textFile("product.txt")

In [4]:
fileRdd.isEmpty()

Py4JJavaError: ignored

In [ ]:
data=fileRdd.map(lambda x:(x.split()[1],1))

In [ ]:
data.take(4)

In [ ]:
cityCount=data.reduceByKey(lambda v1,v2: v1+v2)

In [ ]:
cityCounter=cityCount.collect()

# Totale Price in each City

In [ ]:
cityPriceRdds=fileRdd.map(lambda line : (line.split()[1],float(line.split()[4])))

In [ ]:
cityPriceRdds.take(5)

In [ ]:
cityPriceTotal=cityPriceRdds.reduceByKey(lambda p1,p2: p1+p2)

In [ ]:
result=cityPriceTotal.collect()

In [ ]:
result

In [ ]:
import plotly.express as px

In [ ]:
px.bar(x=[x for x,y in result],y=[y for x,y in result],labels={"x":"city","y":"price"})

In [ ]:
px.bar(x=[x for x,y in cityCounter],y=[y for x,y in cityCounter],labels={"x":"city","y":"number"})

In [ ]:
import pandas as pd
data=pd.read_csv("2020.csv.gz")

In [ ]:
data.head()

In [ ]:
data=data.rename(columns={'AE000041196':"station_code", '20200101':"weather_date", 'TMIN':"element_type", '168':"element_value", 'Unnamed: 4':"measurement_flag", 'Unnamed: 5':"measurement_flag",})


In [ ]:
data.nunique()

# Analys With PYSPARK

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark=SparkSession.builder.master("local[*]").appName("app2").getOrCreate()

In [ ]:
data=spark.read.csv("2020.csv.gz")

In [ ]:
data_v=data.withColumnsRenamed({'_c0':"station_code", '_c1':"weather_date", '_c2':"element_type", '_c3':"element_value", '_c4':"M-FLAG", '_c5':"Q-FLAG","_c6":"S-FLAG","_c7":"OBS-TIME"})

In [ ]:
data_v.show(5)

In [ ]:
tminData=data_v.where("element_type=='TMIN'")

In [ ]:
min=data_v.agg({"element_value":"min"}).collect()[0][0]
print(f"min values of TMIN : {min}")

In [ ]:
data_v.select("element_type").distinct().show()

# Exercice ClassRoom


In [9]:
from pyspark.sql import SparkSession

In [10]:
spark=SparkSession.builder.master("local[*]").appName("exercice").getOrCreate()

In [13]:
data=spark.read.option("multiline",True).json("dataJson")

In [15]:
data.show()

+-----------------+---+---------+------+----+------+------+-------+-----------+--------------+----------+
|          adresse| id|nb_pieces|   nom|numC|prenom|  prix|surface|        tel|          type|     ville|
+-----------------+---+---------+------+----+------+------+-------+-----------+--------------+----------+
|      agdal rabat|  1|        4| ahmed|1111| krami|900000|    110| 0633445530| haut-standing|     rabat|
|      agdal rabat|  2|        3| ahmed|1111| krami|600000|     80| 0633445530|moyen-standing|     rabat|
|maarif casablanca|  3|        4|rachid|2222| Ahmed|900000|    110| 0754352130| haut-standing|casablanca|
|maarif casablanca|  4|        4| hajar|3333|  said|900000|    110|06938473230| haut-standing|casablanca|
|maarif casablanca|  5|        4|  said|4444|belhaj|900000|    110|06938473430| haut-standing|casablanca|
|      agdal rabat|  6|        4| hajar|5555|  said|900000|    110|06938473230|moyen-standing|     rabat|
|      agdal rabat|  7|        3| hajar|5555| 

In [28]:
cityPrice=data.groupby("ville").sum("prix").withColumnsRenamed({"sum(prix)":"prix"})

In [34]:
cityPrice.sort(cityPrice.prix.desc()).show(2)

+----------+-------+
|     ville|   prix|
+----------+-------+
|     rabat|3000000|
|casablanca|2700000|
+----------+-------+



In [50]:
clientCity=data.groupby("ville","numC").count().withColumnsRenamed({"count":"total"})
clientCity.show()

+----------+----+-----+
|     ville|numC|total|
+----------+----+-----+
|     rabat|1111|    2|
|casablanca|4444|    1|
|     rabat|5555|    2|
|casablanca|2222|    1|
|casablanca|3333|    1|
+----------+----+-----+



In [53]:
clientCity.where((clientCity.ville=="rabat") & (clientCity.total>1)).show()

+-----+----+-----+
|ville|numC|total|
+-----+----+-----+
|rabat|1111|    2|
|rabat|5555|    2|
+-----+----+-----+



In [66]:
data.where(data.ville=="casablanca").count()

3